In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

np.random.seed(42)


n = 1000
Z = np.random.binomial(1, 0.5, n)  # Confounder

X = np.random.binomial(1, 0.7 * Z + 0.3 * (1 - Z))

# use X and Z both
Y = 3 * X + 2 * Z + np.random.normal(0, 1, n)  # Outcome

data = pd.DataFrame({'X': X, 'Z': Z, 'Y': Y})
print(data.head())

   X  Z         Y
0  0  0 -0.877983
1  1  1  4.173120
2  0  1  1.773521
3  0  1  2.367366
4  1  0  3.913585


In [3]:
data.groupby(['X', 'Z'])['Y'].mean().shape

(4,)

In [4]:
# average Y
mean_y_x_z = data.groupby(['X', 'Z'])['Y'].mean().unstack() # unstack shapes the result into a table format 
mean_y_x_z, mean_y_x_z.shape 

(Z         0         1
 X                    
 0  0.008959  1.920737
 1  3.095666  5.021463,
 (2, 2))

![](conditional_mean.JPG)

$$
Causal\ Effect = P(Y | do(X=1)) - P(Y | do(X=0))  
$$

In [13]:
mean_y_x_z.loc[1], mean_y_x_z.loc[0]

(Z
 0    3.095666
 1    5.021463
 Name: 1, dtype: float64,
 Z
 0    0.008959
 1    1.920737
 Name: 0, dtype: float64)

In [7]:
data['Z'][:5]

0    0
1    1
2    1
3    1
4    0
Name: Z, dtype: int32

In [12]:
data['Z'].value_counts(), data['Z'].value_counts(normalize=True) 

(Z
 0    503
 1    497
 Name: count, dtype: int64,
 Z
 0    0.503
 1    0.497
 Name: proportion, dtype: float64)

`data['Z'].value_counts(normalize=True) ` tells how frequently each confounder value appears in the dataset.

Product of conditional means(of Z) with respective probabilities in the data provides the conditional probability `(of X=0 or 1)`

In [15]:
print(mean_y_x_z.loc[1] * data['Z'].value_counts(normalize=True))
np.sum(mean_y_x_z.loc[1] * data['Z'].value_counts(normalize=True))

Z
0    1.557120
1    2.495667
dtype: float64


4.052787375035509

In [16]:
mean_y_x_z.loc[0]

Z
0    0.008959
1    1.920737
Name: 0, dtype: float64

In [17]:
print(mean_y_x_z.loc[0] * data['Z'].value_counts(normalize=True))
np.sum(mean_y_x_z.loc[0] * data['Z'].value_counts(normalize=True))

Z
0    0.004506
1    0.954606
dtype: float64


0.9591122608565965

In [4]:

P_Y_do_X1 = np.sum(mean_y_x_z.loc[1] * data['Z'].value_counts(normalize=True))
P_Y_do_X0 = np.sum(mean_y_x_z.loc[0] * data['Z'].value_counts(normalize=True))

causal_effect_backdoor = P_Y_do_X1 - P_Y_do_X0
print(f"Estimated causal effect using backdoor criterion: {causal_effect_backdoor:.2f}")

Estimated causal effect using backdoor criterion: 3.09


In [18]:
import dowhy
from dowhy import CausalModel

model = CausalModel(
    data=data,
    treatment='X',
    outcome='Y',
    common_causes=['Z']
)


# model.view_model(layout="dot")
# plt.show()

identified_estimand = model.identify_effect()
# print("Identified Estimand:", identified_estimand)


causal_estimate = model.estimate_effect(
    identified_estimand,
    method_name="backdoor.linear_regression"
)

print(f"Estimated Causal Effect (DoWhy): {causal_estimate.value:.2f}")

Estimated Causal Effect (DoWhy): 3.09


D:\After\torch\Lib\site-packages\dowhy\causal_estimators\regression_estimator.py:179: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  intercept_parameter = self.model.params[0]


Thus, the estimated causal effect with backdoor from dowhy matches the backdoor causal effect implementation from scratch without dowhy library. 3.09 is the effect observed from both methods!